In [1]:
import pandas as pd
import numpy as np

from scipy.stats import mode
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

import matplotlib.pyplot as plt
import seaborn as sns

# Загрузка и обработка данных

In [2]:
header = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
df = pd.read_csv('data/cars/car.data' ,names=header)

Проверка типов данных признаков

In [3]:
pd.Series([str(df[name].dtype) for name in df.columns]).value_counts()

object    7
Name: count, dtype: int64

Проверка выборки на наличие нулей

In [4]:
percent_missing = df.isnull().sum() * 100 / len(df)
pd.DataFrame({'percent_missing': percent_missing})

,percent_missing
buying,0.0
maint,0.0
doors,0.0
persons,0.0
lug_boot,0.0
safety,0.0
class,0.0


Подсчёт примеров каждого класса

In [5]:
# подсчёт сэмплов каждого класса
def calc_classes_exmpls(dataframe):
    print('Разброс по классам:')
    for cls in dataframe['class'].unique():
        print(f"{cls}: {len(dataframe[dataframe['class'] == cls])}")
calc_classes_exmpls(df)

Разброс по классам:
unacc: 1210
acc: 384
vgood: 65
good: 69


Кодирование категориальных признаков

In [6]:
le = preprocessing.LabelEncoder()
original_classes = np.unique(df['class'].values)
for column_name in df.columns:
    df[column_name] = le.fit_transform(df[column_name])
y = df['class']
X = df.loc[:, df.columns != 'class']

In [7]:
X = X.values
Y = y.values.astype(int)
feature_num = X.shape[1]
classes_num = len(pd.unique(y))
print('Numb of features: ', feature_num)
print('Numb of classes: ', classes_num)

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
print(x_train.shape)
print(y_test.shape)

Numb of features:  6
Numb of classes:  4
(1382, 6)
(346,)


# Реализация алгоритма и построение модели

*Метрики*

In [8]:
def minkowski_distance(x1, x2, p=3):
    minkowski = np.sum(np.abs(x1-x2)**p)**(1/p)
    return minkowski

def euclidean_distance(x1, x2):
    euclidean = np.sqrt(np.sum((x1-x2)**2))
    return euclidean

def manhattan_distance(x1, x2):
    manhattan = np.sum(np.abs(x1-x2))
    return manhattan

*Реализация класса k-Nearest Neighbours*

In [9]:
class KNN_classifier():
    def __init__(self, k, metric=minkowski_distance, p=3) -> None:
        self.k = k
        self.metric = metric
        self.p = p

    def fit(self, X, y):
        self.X = X
        self.y = y
        return self
    
    def predict_one(self, x):
        neighbours = self._get_neighbours(x)
        predicted_class = max(set(neighbours), key=neighbours.count)
        return predicted_class

    def predict(self, x):
        return [self.predict_one(_) for _ in x]

    def score(self, x_test, y_test):
        predictions = []
        for x, y in zip(x_test, y_test):
            y_pred = self.predict_one(x)
            predictions.append(y_pred)
        
        accuracy = np.mean(y_test == predictions)
        return accuracy

    def _get_neighbours(self, x):
        distances = []
        # считаем дистанцию на всей тренировочной выборке
        for x_train, y_train in zip(self.X, self.y):
            distance = self.metric(x_train, x)
            distances.append((distance, y_train))

        # сортируем по возрастанию дистанции
        distances.sort(key=lambda d: d[0])

        # ищем соседей
        neighbours = [distances[i][1] for i in range(self.k)]
        return neighbours


Сравнение точности алгоритмов

In [10]:
class Scorer (object):
  def __init__(self, y_true, y_pred):
    tp, fp, _, fn = self._perf_measure(y_true, y_pred)
    self.tp = tp
    self.fp = fp
    self.fn = fn

  @staticmethod
  def _perf_measure(y_true, y_pred):
      TP = 0
      FP = 0
      TN = 0
      FN = 0

      for i in range(len(y_pred)): 
          if y_true[i] == y_pred[i] == 1:
            TP += 1
          if y_pred[i] == 1 and y_true[i] != y_pred[i]:
            FP += 1
          if y_true[i] == y_pred[i] == 0:
            TN += 1
          if y_pred[i] == 0 and y_true[i] != y_pred[i]:
            FN += 1

      return (TP, FP, TN, FN)

  def get_recall_score(self) -> float:
    return self.tp / (self.tp + self.fn)

  def get_precision_score(self) -> float:
    return self.tp / (self.tp + self.fp)

  def get_f1_score(self) -> float:
    return 2 * self.tp / (2 * self.tp + self.fp + self.fn)


In [11]:
metrics = {
  'Minkowski': minkowski_distance, 
  'Euclidean': euclidean_distance, 
  'Manhattan': manhattan_distance
}
neighbours_count = [3, 5, 8, 10]

In [12]:
def get_scores_df(model: KNN_classifier, neighbours: list, metrics: dict) -> pd.DataFrame:
  scores = pd.DataFrame(columns=metrics.keys(), index=neighbours)

  for k in neighbours:
    row = []
    for metric in metrics.values():
      KNN = model(k, metric=metric).fit(x_train, y_train)
      y_pred = KNN.predict(x_test)
      scorer = Scorer(y_test, y_pred)
      row.append(scorer.get_f1_score())
    scores.loc[k] = row

  return scores

## KNN + Оконный метод

In [13]:
def gaussian_kernel(u):
    return np.exp(-0.5 * u**2) / ((2 * np.pi))

def epanechnikov_kernel(u):
    return 3/4 * (1 - u**2)

In [73]:
class KNN_classifier_Parzen():
    def __init__(self, k, metric=minkowski_distance, p=3, kernel=epanechnikov_kernel) -> None:
        self.k = k
        self.metric = metric
        self.p = p
        self.kernel = kernel

    def fit(self, X, y):
        self.X = X
        self.y = y
        return self
    
    def set_k(self, k):
        self.k = k
    
    def predict_one(self, x):
        neighbours = self._get_neighbours(x)
        predicted_class = self.get_predicted_class(neighbours)
        return predicted_class
    
    def get_predicted_class(self, preds):
        w_arr = np.array(preds)
        res_dict = {}
        for i in range(w_arr.shape[0]):
            cls = w_arr[i][1]
            value = w_arr[i][0]
            if cls not in res_dict.keys():
                res_dict[cls] = value
                continue
            res_dict[cls] += value
        return max(res_dict, key=lambda k: res_dict.get(k))

    def _get_neighbours(self, x):
        distances = []
        # считаем дистанцию на всей тренировочной выборке
        for x_train, y_train in zip(self.X, self.y):
            distance = self.metric(x_train, x)
            distances.append((distance, y_train))
            #print(weight)

        # сортируем по возрастанию дистанции
        distances.sort(key=lambda d: d[0])

        # Применяем окно парзена
        max_dist = distances[self.k+1][0]
        weights = []
        for i, (distance, label) in enumerate(distances):
            if i == self.k:
                break
            weights.append((self.kernel(distance / max_dist), label))
        return weights
    
    def predict(self, x):
        return [self.predict_one(_) for _ in x]

    def score(self, x_test, y_test):
        predictions = []
        for x, y in zip(x_test, y_test):
            y_pred = self.predict_one(x)
            predictions.append(y_pred)
        
        accuracy = np.mean(y_test == predictions)
        return accuracy


In [79]:
neighbours_count = [3, 5, 8, 10]
get_scores_df(KNN_classifier, neighbours_count, metrics)


,Minkowski,Euclidean,Manhattan
3,0.35,0.35,0.35
5,0.580645,0.580645,0.580645
8,0.484848,0.484848,0.545455
10,0.413793,0.413793,0.444444


In [78]:
neighbours_count = [3, 5, 8, 10]
get_scores_df(KNN_classifier_Parzen, neighbours_count, metrics)

,Minkowski,Euclidean,Manhattan
3,0.231884,0.231884,0.231884
5,0.363636,0.363636,0.363636
8,0.774194,0.774194,0.774194
10,0.785714,0.785714,0.785714


In [77]:
get_scores_df(KNeighborsClassifier, neighbours_count, metrics)

,Minkowski,Euclidean,Manhattan
3,0.3,0.3,0.358974
5,0.518519,0.518519,0.592593
8,0.625,0.545455,0.645161
10,0.444444,0.4,0.37037


## LOO

In [80]:
def find_best_k(classifier, k_list):
    results = []
    for k in k_list:
        classifier.set_k(k=k)
        correct_sum = leave_one_out(classifier, x_train, y_train)
        results.append(correct_sum)

    best_k = k_list[(max(enumerate(results), key=(lambda x: x[1])))[0]]
    return best_k

def leave_one_out(model, X : np.array, y : np.array) -> int:
    correct_sum = 0

    for i in range(len(X)):
        x_test_ = X[i]
        y_test_ = y[i]
        X_ = np.delete(X, i, 0)
        y_ = np.delete(y, i, 0)
        model.fit(X_, y_)
        y_pred = model.predict_one(x_test_)
        if y_test_ == y_pred:
            correct_sum += 1

    return correct_sum

In [83]:
k_list = [i for i in range(6, 12, 1)]
classifier = KNN_classifier_Parzen(None, euclidean_distance, kernel=gaussian_kernel)
best_k = find_best_k(classifier, k_list)

get_scores_df(KNN_classifier_Parzen, [best_k], metrics)



,Minkowski,Euclidean,Manhattan
9,0.758621,0.758621,0.758621
